In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer
import torch
from transformers import BertForSequenceClassification, Trainer, TrainingArguments
from transformers import BartForConditionalGeneration, BartTokenizer,AutoTokenizer
df = pd.read_excel('jobDesc.xlsx')

In [7]:
train_texts = df['Experience'].tolist()
train_labels = df['Summarized Experience'].tolist()

train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size = 0.2, random_state = 42)

tokenizer = AutoTokenizer.from_pretrained('facebook/bart-base')


/opt/anaconda3/envs/ml_env/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [8]:
model=BartForConditionalGeneration.from_pretrained('facebook/bart-base')

In [9]:
#tokenize training texts
train_encoding = tokenizer(train_texts, truncation = True, padding = True, max_length = 512, return_tensors = 'pt')
train_inputs = train_encoding['input_ids']
train_attention_mask = train_encoding['attention_mask']

#tokenize training label
train_label_encoding = tokenizer(train_labels, truncation = True, padding = True, max_length = 512, return_tensors = 'pt')
train_label_inputs = train_label_encoding['input_ids']
train_label_attention_mask = train_label_encoding['attention_mask']

 #validation set tokenization
val_encoding = tokenizer(val_texts, truncation = True, padding = True, max_length = 512, return_tensors = 'pt')
val_inputs = val_encoding['input_ids']
val_attention_mask =val_encoding['attention_mask']

val_label_encoding=tokenizer(val_labels,truncation=True,padding=True,max_length=512,return_tensors='pt')
val_label_inputs=val_label_encoding['input_ids']
val_attention_mask=val_label_encoding['attention_mask']

In [10]:
from torch.utils.data import Dataset, DataLoader

In [11]:
class ExperienceDataset(Dataset):
    def __init__(self, input_ids, attention_mask, labels):
        self.input_ids = input_ids
        self.attention_mask = attention_mask
        self.labels = labels

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_mask[idx],
            'labels': self.labels[idx]
        }

In [12]:
# Convert inputs, attention masks, and labels to torch tensors
train_inputs = torch.tensor(train_inputs)
train_attention_mask = torch.tensor(train_attention_mask)
train_label_inputs = torch.tensor(train_label_inputs)

# Create dataset and dataloader
train_dataset = ExperienceDataset(train_inputs, train_attention_mask, train_label_inputs)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

/var/folders/7b/dvnb1fs51g51bqngddg9d9fc0000gn/T/ipykernel_18934/701493472.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_inputs = torch.tensor(train_inputs)
/var/folders/7b/dvnb1fs51g51bqngddg9d9fc0000gn/T/ipykernel_18934/701493472.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_attention_mask = torch.tensor(train_attention_mask)
/var/folders/7b/dvnb1fs51g51bqngddg9d9fc0000gn/T/ipykernel_18934/701493472.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_label_inputs = torch.tensor(train_label_inputs)


In [16]:
from transformers import AdamW
from tqdm import tqdm

# Initialize AdamW optimizer
optimizer = AdamW(model.parameters(), lr=1e-5,weight_decay=0.01)

# Training loop
epochs = 3
device = torch.device("cpu")
model = model.to(device)

for epoch in range(epochs):
    model.train()
    loop = tqdm(train_loader, leave=True)

    for batch in loop:
        optimizer.zero_grad()

        # Move batch to device
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        # Forward pass (BART expects input_ids and labels for conditional generation)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)

        # Loss
        loss = outputs.loss
        loss.backward()

        # Optimizer step
        optimizer.step()

        # Update progress bar
        loop.set_description(f"Epoch {epoch}")
        loop.set_postfix(loss=loss.item())


Epoch 2: 100%|██████████| 22/22 [01:30<00:00,  4.11s/it, loss=nan]


In [15]:
model.eval()
text_evaluation="Leave Application Processing System (Web application) Dec 2023 Team leader Singapore System Design: Used the MVC design pattern to build the system architecture. Develpoment:Developed leave application processing system with java, spring, MySql, and h2 database, enabling leave requests, historical tracking, and administrative management. Github:https://github.com/Zxuanji/Leave-Application-Processing-System"
test_encoding = tokenizer(text_evaluation, return_tensors="pt", truncation=True, padding=True, max_length=512)
test_input_ids = test_encoding['input_ids'].to(device)

generated_ids = model.generate(test_input_ids, max_length=150, num_beams=4, early_stopping=True)
generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
print(generated_text)

 the the the
